# DreamJobber

**Tech Edition**

---

**Process**
1. Clean text
2. Bag of Words
3. LDA model (Latent Dirichlet allocation)
4. Fine tune LDA model
5. Define Topics from LDA model
6. Create df of document probabilities
6. Classification model

---

**Import Necessary Libraries**

In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
from dreamjobber_tech.recommend import *
from functions import *
import pickle

#lda model evaluatoin with coherence
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

from sklearn.neighbors import NearestNeighbors

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

---

**Load Data**

In [3]:
df_1 = pd.read_json('data/dice_jobs_1.json', lines=True)
df_2 = pd.read_json('data/dice_jobs_2.json', lines=True)
df_3 = pd.read_json('data/dice_jobs_3.json', lines=True)
df_4 = pd.read_json('data/dice_jobs_4.json', lines=True)
df_5 = pd.read_json('data/dice_jobs_5.json', lines=True)
df_6 = pd.read_json('data/dice_jobs_6.json', lines=True)

In [4]:
#concat into one df
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], ignore_index=True, sort=True)

In [5]:
df.head()

,job_description,job_title
0,NaN,UI Lead/Architect
1,NaN,Web Application Architect
2,NaN,Senior DataStage Developer
3,NaN,Hadoop Administrator
4,NaN,UX Visual Designer


In [6]:
#check for missing values
df.isna().sum()

job_description    6524
job_title          5484
dtype: int64

In [7]:
#looks like there are rows that have no job description
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27016 entries, 0 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          21532 non-null object
dtypes: object(2)
memory usage: 422.2+ KB


In [8]:
#drop rows with no job descriptions
df = df.dropna()

In [9]:
#sanity check, looks good
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20492 entries, 9 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          20492 non-null object
dtypes: object(2)
memory usage: 480.3+ KB


In [10]:
df.head()

,job_description,job_title
9,[2+ years of experience developing Java / J2EE...,Java Developer (Sign-On BONUS!)
10,"[Passion for technology and learning, a natura...","iOS Developer - Mobile Rate: Open, Duration: 1..."
17,[We enjoy approved IT vendor status with sever...,"EUC Engineer, Rate-Open, Duration: 18 Months"
18,[We enjoy approved IT vendor status with sever...,"Software Developer - RPG, Rate-Open, Duration:..."
19,[SME in Linux Operating system with Strong Vir...,Sr. Linux Consultant with Weblogic exp


In [11]:
#need to remove brackets from job_description
df['job_description'] = df['job_description'].map(remove_brackets)

In [12]:
#remove '\\n' and replace with ','
df['job_description'] = df['job_description'].map(lambda x: x.replace('\\n', ','))

In [13]:
#lowercase job_description text before applying stopwords
df['job_description'] = df['job_description'].map(lambda x: x.lower())

In [14]:
#lowercase job_title text before cleaning
df['job_title'] = df['job_title'].map(lambda x: x.lower())

In [15]:
#remove punctuation from job_title
df['job_title'] = df['job_title'].map(remove_punctuation)

In [16]:
#remove stop words from job_title
df['job_title'] = df['job_title'].map(remove_stop_words)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20492 entries, 9 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          20492 non-null object
dtypes: object(2)
memory usage: 480.3+ KB


In [18]:
#drop any duplicates
df = df.drop_duplicates()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18704 entries, 9 to 27015
Data columns (total 2 columns):
job_description    18704 non-null object
job_title          18704 non-null object
dtypes: object(2)
memory usage: 438.4+ KB


In [20]:
#reset the index
df = df.reset_index(drop=True)

In [21]:
#check to see if there are any job titles that have been removed via stopwords, I will want to remove these rows
#because the job titles were not real job titles
df.loc[df['job_title']=='']

,job_description,job_title
1526,'',
2508,'robert half technology is looking for an expe...,
13230,'someone who has spent 8-10 years in insurance...,
13569,"'my client is looking for sap fico in houston,...",
14136,"'share your resume on click here to apply', 'j...",
14157,"'role : fico with s4 hana.,location : manitowo...",
15118,"'hello all,', '', '', '', '', '', '', '', '', ...",
16508,"'job description:', 'leidos is looking for a j...",


In [22]:
df = df.drop(df.index[[1526, 2508, 13230, 13569, 14136, 14157, 15118, 16508]])

In [23]:
#sanity check
df.loc[df['job_title']=='']

,job_description,job_title


In [24]:
#reset index one last time
df = df.reset_index(drop=True)
df.head()

,job_description,job_title
0,'2+ years of experience developing java / j2ee...,java developer
1,"'passion for technology and learning, a natura...",ios developer mobile
2,'we enjoy approved it vendor status with sever...,euc engineer
3,'we enjoy approved it vendor status with sever...,software developer rpg
4,'sme in linux operating system with strong vir...,linux consultant with weblogic exp


---

## Text Cleaning

1. Tokenize
2. Remove words with fewer than 3 characters
3. Remove stop words
4. Normalize words (Lemmatize and Stem)

**Test the functions on one row of text**

In [25]:
stemmer = SnowballStemmer('english')

In [26]:
text_sample = df[df.index == 41].values[0][0]

print('original text: ')
words = []
for word in text_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized text: ')
print(preprocess(text_sample))

original text: 
["'8-10", 'years', 'of', 'relevant', "experience.',", "'expert", 'knowledge', 'of', 'java', 'and', 'object', 'oriented', 'design', "patterns.',", "'expert", 'knowledge', 'of', 'dependency', 'injection.', 'experience', 'with', 'spring', 'or', 'ejb3', 'or', 'guice', 'is', 'a', 'must.', 'experience', 'with', 'guice', 'is', "preferred.',", "'strong", 'in', 'logic,', 'algorithm', 'and', 'data', 'strucutures', '-', 'problem', "solving',", "'experience", 'with', 'oracle', 'or', 'mysql', 'and', 'writing', "sqls.',", "'experience", 'in', 'angular', '2+', 'and', "typescript',", "'expert", 'knowledge', 'of', 'junit,', 'mockito', 'or', 'other', 'unit', 'testing', 'best', "practices.',", "'4+", 'years', 'of', 'experience', 'developing', 'and', 'deploying', 'applications', 'on', 'unix/linux', "environments',", "'experience", 'with', 'orm', 'such', 'as', 'hibernate', "orm',", "'experience", 'with', 'web', 'development', 'or', 'rest/', 'soap', 'web', 'services', 'development', 'and', '

**Apply cleaning functions to job_description**

In [27]:
#apply function and display first 5 rows
processed_text = df['job_description'].map(preprocess)
processed_text[:5]

0                                 [develop, java, web]
1    [passion, technolog, learn, natur, curios, lov...
2    [enjoy, approv, vendor, status, lead, compani,...
3    [enjoy, approv, vendor, status, lead, compani,...
4    [sme, linux, oper, strong, virtual, knowledg, ...
Name: job_description, dtype: object

---

## Bag of Words

In [28]:
#I'll use bag of words to extract features from text for use in modeling

In [29]:
dictionary = gensim.corpora.Dictionary(processed_text)

In [30]:
#check the length before I filter out the extremes
len(dictionary)

24548

In [31]:
dictionary.filter_extremes(no_below=25, no_above=0.5, keep_n=100000)

In [32]:
#check length after filtering out extremes
len(dictionary)

3753

In [33]:
#bow2doc: counts the number of occurrences of each distinct word, 
#converts the word to its integer word id and returns the result as a sparse vector

bow2doc_corpus = [dictionary.doc2bow(text) for text in processed_text]

---

## Find optimal number of topics

In [ ]:
#model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=bow2doc_corpus,
 #                                                       texts=processed_text, start=5, limit=40, step=5)

In [ ]:
#import matplotlib.pyplot as plt
#limit=40; start=5; step=5;
#x = range(start, limit, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Number of Topics")
#plt.ylabel("Coherence score")
#plt.show()

---

## LDA model with Bag of Words

In [42]:
lda_model = gensim.models.LdaMulticore(bow2doc_corpus, 
                                       num_topics=9, 
                                       id2word=dictionary, 
                                       passes=50, 
                                       workers=4,
                                      chunksize=500)


In [43]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.068*"secur" + 0.020*"inform" + 0.018*"cloud" + 0.016*"system" + 0.016*"architectur" + 0.013*"engin" + 0.011*"oper" + 0.010*"network" + 0.010*"implement" + 0.009*"provid"
Topic: 1 
Words: 0.016*"engin" + 0.016*"success" + 0.012*"protect" + 0.012*"receiv" + 0.011*"status" + 0.010*"look" + 0.009*"client" + 0.009*"innov" + 0.009*"group" + 0.009*"solut"
Topic: 2 
Words: 0.054*"test" + 0.020*"requir" + 0.016*"function" + 0.015*"document" + 0.014*"integr" + 0.013*"technic" + 0.013*"user" + 0.013*"softwar" + 0.013*"process" + 0.013*"implement"
Topic: 3 
Words: 0.028*"network" + 0.019*"system" + 0.014*"server" + 0.013*"oper" + 0.012*"technic" + 0.012*"troubleshoot" + 0.012*"environ" + 0.012*"softwar" + 0.012*"administr" + 0.011*"window"
Topic: 4 
Words: 0.030*"market" + 0.019*"product" + 0.018*"digit" + 0.014*"creativ" + 0.013*"content" + 0.011*"user" + 0.010*"creat" + 0.009*"sale" + 0.008*"web" + 0.008*"custom"
Topic: 5 
Words: 0.037*"project" + 0.016*"abil" + 0.013*"process

**Pickled LDA model results**

In [44]:
#pickle.dump(lda_model, open('pickled_models/lda_model.pkl', 'wb'))
pickled_lda = pickle.load(open('pickled_models/lda_model.pkl', 'rb'))

In [41]:
for idx, topic in pickled_lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.120*"data" + 0.038*"secur" + 0.022*"databas" + 0.018*"analyst" + 0.018*"sql" + 0.016*"model" + 0.015*"report" + 0.015*"inform" + 0.014*"system" + 0.014*"analyt"
Topic: 1 
Words: 0.020*"engin" + 0.018*"status" + 0.014*"protect" + 0.014*"receiv" + 0.011*"origin" + 0.010*"inform" + 0.010*"qualifi" + 0.009*"regard" + 0.009*"program" + 0.007*"provid"
Topic: 2 
Words: 0.039*"project" + 0.014*"abil" + 0.012*"process" + 0.012*"technic" + 0.011*"plan" + 0.011*"solut" + 0.010*"lead" + 0.009*"ensur" + 0.009*"product" + 0.008*"requir"
Topic: 3 
Words: 0.017*"custom" + 0.017*"abil" + 0.017*"technic" + 0.015*"issu" + 0.015*"problem" + 0.013*"provid" + 0.012*"user" + 0.009*"system" + 0.009*"softwar" + 0.009*"environ"
Topic: 4 
Words: 0.015*"market" + 0.012*"client" + 0.010*"product" + 0.010*"look" + 0.009*"digit" + 0.009*"help" + 0.007*"creativ" + 0.007*"compani" + 0.007*"industri" + 0.007*"custom"
Topic: 5 
Words: 0.068*"test" + 0.023*"softwar" + 0.023*"requir" + 0.017*"document" 

---

**LDA model evaluation**

In [ ]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_text, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Compute Coherence Score using UMass
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_text, dictionary=dictionary, coherence="u_mass")
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
#!pip install pyLDAvis

In [ ]:
##visualize the topics in order to better label 
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=bow2doc_corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)


In [ ]:
#show topics and descriptions
df_topic_sents_keywords = show_topics_sentences(ldamodel=pickled_lda, corpus=bow2doc_corpus, texts=df['job_description'])


df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


In [ ]:
df_dominant_topic.head()

---

**Create df for topic scores for each jobtitle**

In [ ]:
topic_vecs = []
for i in range(len(bow2doc_corpus)):
    top_topics = pickled_lda.get_document_topics(bow2doc_corpus[i], minimum_probability=0.0)
    #i in range(amount of topics)
    topic_vec = [top_topics[i][1] for i in range(9)]
    topic_vecs.append(topic_vec)

In [ ]:
df_topic_vecs = pd.DataFrame(topic_vecs)
df_topic_vecs.head(10)

In [ ]:
#name columns for df

col_names=['Leadership', 'WebDev', 'Cloud Computing', 'Computer Network', 
           'Database Admin', 'Software/App Dev', 'Analyst', 'Security', 'Computer Support']



df_topic_vecs.columns = col_names
df_topic_vecs.head()

---

---

# Nearest Neighbors

In [ ]:
#next step merge with original df of job titles and job descriptions
#pickle the merged df

In [ ]:
df_final = pd.merge(df, df_topic_vecs,left_index=True, right_index=True)

In [ ]:
#pickle.dump(df_final, open('pickled_models/df_final.pkl', 'wb'))
pickled_df_final = pickle.load(open('pickled_models/df_final.pkl', 'rb'))
pickled_df_final.head()

In [ ]:
topics = pickled_df_final.drop(['job_description', 'job_title'], axis=1)
jobs = pickled_df_final['job_title']

In [ ]:
#pickle.dump(jobs, open('pickled_models/jobs.pkl', 'wb'))
jobs = pickle.load(open('pickled_models/jobs.pkl', 'rb'))

In [ ]:
nearest_neighbor = NearestNeighbors(n_neighbors=50)
nearest_neighbor.fit(topics)

In [ ]:
#pickle.dump(nearest_neighbor, open('pickled_models/nn_model.pkl', 'wb'))
nearest_neighbor = pickle.load(open('pickled_models/nn_model.pkl', 'rb'))

---

**Make Recommendations**

In [ ]:
show_to_user(nearest_neighbor, jobs)


In [ ]:
## next-steps
#1.clean job-titles!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#2.recommendations return unique list of top 10
#3.seperate functions (textcleaning.py, recommend.py)
#4.flask, level up some links from your recommendations to jobs
